<a href="https://colab.research.google.com/github/aamirkhanov/mu2e_crv/blob/main/DataFrameMakerCRY5_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!apt-get -qq install krb5-user
!wget https://authentication.fnal.gov/krb5conf/Linux/krb5.conf
import os
os.environ['KRB5_CONFIG'] = 'krb5.conf'

^C
--2022-08-25 14:41:15--  https://authentication.fnal.gov/krb5conf/Linux/krb5.conf
Resolving authentication.fnal.gov (authentication.fnal.gov)... 131.225.105.240, 2620:6a:0:105::105:240
Connecting to authentication.fnal.gov (authentication.fnal.gov)|131.225.105.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17486 (17K) [text/plain]
Saving to: ‘krb5.conf.1’

krb5.conf.1         100%[===================>]  17.08K  --.-KB/s    in 0.03s   

2022-08-25 14:41:15 (502 KB/s) - ‘krb5.conf.1’ saved [17486/17486]



In [90]:
!kinit amirkhan@FNAL.GOV

Password for amirkhan@FNAL.GOV: 
kinit: Password read interrupted while getting initial credentials


In [ ]:
#!scp -o StrictHostKeyChecking=no -o GSSAPIAuthentication=yes -o GSSAPIDelegateCredentials=yes amirkhan@mu2egpvm03.fnal.gov:/mu2e/data/users/oksuzian/trk.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0.root .
#!scp -o StrictHostKeyChecking=no -o GSSAPIAuthentication=yes -o GSSAPIDelegateCredentials=yes amirkhan@mu2egpvm03.fnal.gov:/mu2e/data/users/oksuzian/nts.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0.tka .

In [92]:
#!rm -f drive/MyDrive/root_files_cry5/*v1_0_v2*
!ls -lrth drive/MyDrive/root_files_cry5/
#!ls -lrth
#!cp trk.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0_v2.h5 drive/MyDrive/root_files_cry5/

total 11G
-rw------- 1 root root  38M Apr 25 21:18 ana.mu2e.CosmicCRYTriggered.v5hiMinRangeRegionCuts.root
-rw------- 1 root root 4.5G May  4 03:23 cry4df.h5
-rw------- 1 root root 157M May 19 14:18 trkanaCosmicCRYonRecoTriggered010722.root
-rw------- 1 root root  72M May 19 14:19 cry5df_perfect_v1_0.h5
-rw------- 1 root root 436M May 20 18:14 trk.mu2e.CosmicCRYloOnSpillSignal.MDC2020r_perfect_v1_0.root
-rw------- 1 root root 1.2G May 22 02:16 trk.mu2e.CeEndpointMix1BBSignal.MDC2020r_perfect_v1_0.root
-rw------- 1 root root 325M May 22 16:48 cry5df.h5
-rw------- 1 root root 1.4G May 26 15:27 trk.mu2e.CeEndpointMix1BBSignal.MDC2020r_perfect_v1_0.h5
-rw------- 1 root root 1.5G Jun  2 04:17 trk.mu2e.CeEndpointMix2BBSignal.MDC2020r_perfect_v1_0.root
-rw------- 1 root root 457M Jun  2 16:29 trk.mu2e.CeEndpointMix2BBSignal.MDC2020r_perfect_v1_0.h5
-rw------- 1 root root 140M Jun  6 13:48 trk.mu2e.CosmicCRYhiOnSpillSignal.MDC2020r_perfect_v1_0.root
-rw------- 1 root root 218M Jun  6 13:52 trk

# Init and define function

In [1]:
import sys
import matplotlib.pyplot as plt
%pip -q install uproot3
%pip -q install --upgrade tables
import uproot3
import pandas as pd
import numpy as np

DATASET='CE2S'

if DATASET=='CRY4':
  h5file='drive/MyDrive/root_files_cry5/cry4df.h5'
  files_root = {                  
        'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.092020_7000_lo.root': [7000, 0],
        'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.123020_7000_lo.root': [7000, 0],
        'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.092020_9000_lo.root': [9000, 0],
        'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.123020_9000_lo.root': [9000, 0],         
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_11000_lo.root': [11000, 0],
        'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.123020_11000_lo.root': [11000, 0],         
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_13000_lo.root': [13000, 0],
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.123020_13000_lo.root': [13000, 0],
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_15000_lo.root': [15000, 0],
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.123020_15000_lo.root': [15000, 0],        
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_17000_lo.root': [17000, 0],
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.123020_17000_lo.root': [17000, 0],
        
        'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.092020_7000_hi.root': [7000, 1],
        'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.092020_9000_hi.root': [9000, 1],
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_11000_hi.root': [11000, 1],
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_13000_hi.root': [13000, 1],
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_15000_hi.root': [15000, 1],
        'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_17000_hi.root': [17000, 1],
        }
elif DATASET=='CRY5':
  h5file='drive/MyDrive/root_files_cry5/trk.mu2e.CosmicCRYOnSpillSignal.MDC2020r_perfect_v1_0.h5'
  files_root={                  
    'drive/MyDrive/root_files_cry5/trk.mu2e.CosmicCRYloOnSpillSignal.MDC2020r_perfect_v1_0.root': [17000, 0],
    'drive/MyDrive/root_files_cry5/trk.mu2e.CosmicCRYhiOnSpillSignal.MDC2020r_perfect_v1_0.root': [17000, 1],
    }
elif DATASET=='CE1B':
  h5file='drive/MyDrive/root_files_cry5/trk.mu2e.CeEndpointMix1BBSignal.MDC2020r_perfect_v1_0.h5'
  files_root={                  
    'drive/MyDrive/root_files_cry5/trk.mu2e.CeEndpointMix1BBSignal.MDC2020r_perfect_v1_0.root': [17000, -1],
  }
elif DATASET=='CE2B':
  h5file='drive/MyDrive/root_files_cry5/trk.mu2e.CeEndpointMix2BBSignal.MDC2020r_perfect_v1_0.h5'
  files_root={                  
    'drive/MyDrive/root_files_cry5/trk.mu2e.CeEndpointMix2BBSignal.MDC2020r_perfect_v1_0.root': [17000, -1],
  }
elif DATASET=='CE2S':
  #h5file='drive/MyDrive/root_files_cry5/trk.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0.h5'
  h5file='drive/MyDrive/root_files_cry5/trk.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0_v2.h5'
  #h5file='trk.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0_v2.h5'
  files_root={      
    'nts.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0.tka' : [17000, 1],              
    #'drive/MyDrive/root_files_cry5/trk.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0.root': [17000, 1],
  }
else:
  raise ValueError('Unknown dataset')

# This allows you to run on Colaboratory and load files from your Google Drive
try:
    from google.colab import drive
    drive.mount("/content/drive")
    main_path = '/content/drive/MyDrive/'
except:
    main_path = ''
sys.path.append(main_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def default_flatname(branchname, fieldname, index):
    """This replaces the dot in the column names with underscores"""
    out = branchname.replace(".", "_")
    if not isinstance(branchname, str):
        out = branchname.decode("utf-8")
    if fieldname is not None:
        fieldname.replace(".", "_")
        out += "_" + fieldname
    if index != ():
        out += "[" + "][".join(str(x) for x in index) + "]"
    return out

In [34]:
file = uproot3.open(list(files_root.keys())[0])
trkana = file["TrkAnaNeg"]["trkana"]
#trkana.pandas.df(['debestcrv.*'], flatname=default_flatname, flatten=True)
trkana.pandas.df(['debestcrv*'])

#list(trkana.keys())

""
entry
0
1
2
3
4
...
62656
62657
62658


In [95]:
'''
file = uproot3.open(list(files_root.keys())[0])
trkana = file["TrkAnaNeg"]["trkana"]
df_eve = trkana.pandas.df(['evtinfo.*', 'evtinfomc.*', 'de.*', 'deent*', 'dequal.*', 'demc.*', 'demcgen.*', 'demcpri.*', 'ue.*', 'crvsummary.*', 'crvsummarymc.*', '_*', "detrkpid.*"], flatname=default_flatname, flatten=True)
df_crv = trkana.pandas.df(['crvinfo.*', 'crvinfomc.*','bestcrv*'], flatname=default_flatname, flatten=True)
df_crv.columns = df_crv.columns.str.replace('bestcrv', 'crvinfo')
df_eve["deent_mom"] = pow(pow(df_eve["deent_mom_fCoordinates_fX"],2) + pow(df_eve["deent_mom_fCoordinates_fY"],2) + pow(df_eve["deent_mom_fCoordinates_fZ"],2),0.5)
df_eve["demcpri_mom"] = pow(pow(df_eve["demcpri_mom_fCoordinates_fX"],2) + pow(df_eve["demcpri_mom_fCoordinates_fY"],2) + pow(df_eve["demcpri_mom_fCoordinates_fZ"],2),0.5)

pd.merge(df_eve, df_crv, how="left", on='entry')
df = df_eve.merge(df_crv, how="left", on="entry")
df.columns = df.columns.str.replace('__', '_')
df.columns = df.columns.str.replace('_fCoordinates_f', '')
df["crvinfo_dT"] = df["crvinfo_timeWindowStart"] - df["de_t0"]

list(df)
'''

'\nfile = uproot3.open(list(files_root.keys())[0])\ntrkana = file["TrkAnaNeg"]["trkana"]\ndf_eve = trkana.pandas.df([\'evtinfo.*\', \'evtinfomc.*\', \'de.*\', \'deent*\', \'dequal.*\', \'demc.*\', \'demcgen.*\', \'demcpri.*\', \'ue.*\', \'crvsummary.*\', \'crvsummarymc.*\', \'_*\', "detrkpid.*"], flatname=default_flatname, flatten=True)\ndf_crv = trkana.pandas.df([\'crvinfo.*\', \'crvinfomc.*\',\'bestcrv*\'], flatname=default_flatname, flatten=True)\ndf_crv.columns = df_crv.columns.str.replace(\'bestcrv\', \'crvinfo\')\ndf_eve["deent_mom"] = pow(pow(df_eve["deent_mom_fCoordinates_fX"],2) + pow(df_eve["deent_mom_fCoordinates_fY"],2) + pow(df_eve["deent_mom_fCoordinates_fZ"],2),0.5)\ndf_eve["demcpri_mom"] = pow(pow(df_eve["demcpri_mom_fCoordinates_fX"],2) + pow(df_eve["demcpri_mom_fCoordinates_fY"],2) + pow(df_eve["demcpri_mom_fCoordinates_fZ"],2),0.5)\n\npd.merge(df_eve, df_crv, how="left", on=\'entry\')\ndf = df_eve.merge(df_crv, how="left", on="entry")\ndf.columns = df.columns.str.rep

In [11]:
def make_df(trkana, tag_high, lyield):

  if DATASET=='CRY4':
    df_eve = trkana.pandas.df(['evtinfo.', 'de', 'deent', 'dequal', 'demc', 'demcgen', 'trigbits', 'ue', 'crvsummary.*', '_*', "detrkpid.*"], flatname=default_flatname, flatten=True)
    df_crv = trkana.pandas.df(['crvinfo.*', 'crvinfomc.*', 'bestcrv*'], flatname=default_flatname, flatten=True)
    df_crv = df_crv[df_crv.index.get_level_values('subentry') == df_crv["bestcrv"]]
    df_eve["deent_maxr"] = df_eve["deent_d0"]+2.0/df_eve["deent_om"]
    df_eve["is_triggered"] = (df_eve["trigbits"].values & 0x208) > 0
  else: 
    df_eve = trkana.pandas.df(['evtinfo.*', 'evtinfomc.*', 'de.*', 'deent*', 'dequal.*', 'demc.*', 'demcgen.*', 'demcpri.*', 'ue.*', 'crvsummary.*', 'crvsummarymc.*', '_*', "detrkpid.*"], flatname=default_flatname, flatten=True)
    #df_crv = trkana.pandas.df(['crvinfo.*', 'crvinfomc.*','bestcrv*'], flatname=default_flatname, flatten=True)
    df_crv = trkana.pandas.df(['crvinfomc.*', 'debestcrv*'], flatname=default_flatname, flatten=True)
    df_crv.columns = df_crv.columns.str.replace('debestcrv', 'crvinfo')
    df_eve["deent_mom"] = pow(pow(df_eve["deent_mom_fCoordinates_fX"],2) + pow(df_eve["deent_mom_fCoordinates_fY"],2) + pow(df_eve["deent_mom_fCoordinates_fZ"],2),0.5)
    df_eve["demcpri_mom"] = pow(pow(df_eve["demcpri_mom_fCoordinates_fX"],2) + pow(df_eve["demcpri_mom_fCoordinates_fY"],2) + pow(df_eve["demcpri_mom_fCoordinates_fZ"],2),0.5)

  pd.merge(df_eve, df_crv, how="left", on='entry')
  df = df_eve.merge(df_crv, how="left", on="entry")
  df.columns = df.columns.str.replace('__', '_')
  df.columns = df.columns.str.replace('_fCoordinates_f', '')

  df['is_high'] = tag_high
  df['lyield'] = int(lyield)
  df["crvinfo_dT"] = df["crvinfo_timeWindowStart"] - df["de_t0"]
  df["crvinfo_dTe"] = df["crvinfo_timeWindowEnd"] - df["de_t0"]
  df = df.fillna(-999999)
  return df

In [4]:
def combine_df(files):
    
  df = pd.DataFrame()
  for f in files:
      print("Reading file", f, files[f][0], files[f][1])
      file = uproot3.open(f)
      trkana = file["TrkAnaNeg"]["trkana"]
      df_ = make_df(trkana, files[f][1], files[f][0])
      df_ = df_.filter(regex=var_str)
      df = df.append(df_)
  df = df.loc[:,~df.columns.duplicated()]  # delete duplicate colummns
  return df    

#Variables list to save in df

In [5]:
var_list=[
 'evtinfo_eventid',
 'evtinfo_subrunid',
 'evtinfomc_nprotons',
 'de_status',
 'de_pdg',
 'de_nhits',
 'de_t0',
 'deent_mom',
 'deent_d0',
 'deent_p0',
 'deent_z0',
 'deent_td',
 'dequal_TrkPID',
 'dequal_TrkQual',
 'detrkpid',
 'demc_pdg',
 'demc_gen',
 'demcgen_pdg',
 'demcgen_gen',
 'demcgen_posx',
 'demcgen_posy',
 'demcgen_posz',
 'demcgen_momx',
 'demcgen_momy',
 'demcgen_momz',
 'demcpri_posX',
 'demcpri_posY',
 'demcpri_posZ',
 'demcpri_posZ',
 'demcpri_pdg',
 'demcpri_mom',
 'trigbits',
 'ue_status',
 'ue_nhits',
 '_totalPEs',
 '_nHitCounters',
 '_totalEnergyDeposited',
 '_x',
 '_y',
 '_z',
 '_pdgId',
 '_crvSectorNumber',
 '_crvSectorType',
 'crvsummary_totalPEs',
 'crvsummary_nHitCounters',
 'crvsummarymc_totalEnergyDeposited',
 'crvsummarymc_x',
 'crvsummarymc_y',
 'crvsummarymc_z',
 'crvsummarymc_pdgId',
 'crvsummarymc_crvSectorNumber',
 'crvsummarymc_crvSectorType',
 'crvinfo_crvSectorType',
 'crvinfo_x',
 'crvinfo_y',
 'crvinfo_z',
 'crvinfo_PEs',
 'crvinfo_timeWindowStart',
 'crvinfo_timeWindowEnd',
 'crvinfo_angle',
 'crvinfo_nCoincidenceHits',
 'crvinfo_nCoincidenceLayers',
 'crvinfomc',
 'bestcrv',
 'is_high',
 'lyield',
 'crvinfo_dT',
 'crvinfo_dTe',
 'deent_maxr',
 'is_triggered']

var_str='('
for var in var_list:
  if var == "detrkpid":
    var_str=var_str+var+'|'
  elif var == 'crvinfomc':
    var_str=var_str+var+'|'
  else: 
    var_str=var_str+var+'$|'

var_str=var_str[:-1]+')'

# Save dataframe

In [12]:
df = combine_df(files_root)
df.to_hdf(h5file, key=DATASET)

Reading file nts.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0.tka 17000 1


KeyError: ignored

In [10]:
list(df)

['evtinfo_eventid',
 'evtinfo_subrunid',
 'evtinfomc_nprotons',
 'de_status',
 'de_pdg',
 'de_nhits',
 'de_t0',
 'deent_d0',
 'deent_maxr',
 'deent_td',
 'dequal_TrkPID',
 'dequal_TrkQual',
 'demc_pdg',
 'demc_gen',
 'demcpri_pdg',
 'demcpri_posX',
 'demcpri_posY',
 'demcpri_posZ',
 'ue_status',
 'ue_nhits',
 'crvsummary_totalPEs',
 'crvsummary_nHitCounters',
 'crvsummarymc_totalEnergyDeposited',
 'crvsummarymc_nHitCounters',
 'crvsummarymc_x',
 'crvsummarymc_y',
 'crvsummarymc_z',
 'crvsummarymc_crvSectorNumber',
 'crvsummarymc_crvSectorType',
 'crvsummarymc_pdgId',
 '_crvSectorType',
 '_x',
 '_y',
 '_z',
 '_pdgId',
 'detrkpid_DeltaE',
 'detrkpid_ClusterLength',
 'detrkpid_RPOCA',
 'detrkpid_TrackDirection',
 'detrkpid_DeltaT',
 'detrkpid_mvaout',
 'detrkpid_mvastat',
 'detrkpid_disk0frad',
 'detrkpid_disk1frad',
 'detrkpid_disk0brad',
 'detrkpid_disk1brad',
 'deent_mom',
 'demcpri_mom',
 'crvinfomc_valid',
 'crvinfomc_pdgId',
 'crvinfomc_primaryPdgId',
 'crvinfomc_primaryE',
 'crvinf

#Load dataframe

In [ ]:
df = pd.read_hdf(h5file, mode='r', key=DATASET).query('lyield==17000')

In [ ]:
for l in range(7000,17001,2000):
  plt.hist(df.query('is_high==1 & lyield==%i' % l)['crvinfo_PEs'], bins=100, range=(0, 1000), histtype='step');
plt.figure()
for l in range(7000,17001,2000):
  plt.hist(df.query('is_high==0 & lyield==%i' % l)['crvinfo_PEs'], bins=100, range=(0, 1000), histtype='step');
plt.figure()
plt.hist(df.query('is_high==0 & lyield==%i' % 17000)['crvinfomc_z'], bins=100, range=(-5000, 20000), histtype='step');
plt.hist(df.query('is_high==1 & lyield==%i' % 17000)['crvinfomc_z'], bins=100, range=(-5000, 20000), histtype='step');

In [ ]:
cut_tmp = 'lyield==%i' % 17000
plt.plot(df.query(cut_tmp)['crvinfomc_z'], df.query(cut_tmp)['crvinfomc_x'], '.', markersize=0.5);

In [ ]:
!ls -l

In [ ]:
!cp trk.mu2e.CosmicCRYhiSigFiltMix2BBSignal.MDC2020s_perfect_v1_0.h5 drive/MyDrive/root_files_cry5/